In [13]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *

In [14]:
client = QdrantClient(URL,port=6333)

In [15]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [16]:
from llama_index.llms import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [17]:
from llama_index.embeddings import FastEmbedEmbedding

embed_model = FastEmbedEmbedding()

In [18]:
from llama_index import VectorStoreIndex,ServiceContext
from llama_index.storage import StorageContext

In [19]:
service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model)

In [20]:
index = VectorStoreIndex.from_vector_store(service_context=service_context,vector_store=vector_store)

In [21]:
query_engine = index.as_query_engine()

In [22]:
query = "What are Arthopods?"

In [23]:
response = query_engine.query(query)

In [24]:
print(response.get_formatted_sources())
print("query was:", query)
print("answer was:", response)

> Source (Doc id: 98b6c8c7-92d3-490f-9bc9-b6566c6c3a43): 54
BIOLOGY
14. Match the following:
(a) Operculum
(i)
Ctenophora
(b) Parapodia
(ii)
Mollusca
(c) ...

> Source (Doc id: 886448d3-f242-41b8-a6d3-bc8d293e4def): 44
BIOLOGY
4.2.7
Phylum – Arthropoda
This is the largest phylum of Animalia which
includes insect...
query was: What are Arthopods?
answer was: Arthropods are the largest phylum of Animalia which includes insects. They have an organ-system level of organisation, are bilaterally symmetrical, triploblastic, segmented and coelomate animals. Their body is covered by a chitinous exoskeleton and consists of head, thorax and abdomen. They have jointed appendages, respiratory organs, and sensory organs like antennae, eyes, and balancing organs. Excretion takes place through malpighian tubules and they are mostly dioecious. Fertilisation is usually internal and they are mostly oviparous. Examples of economically important arthropods include Apis (Honey bee), Bombyx (Silkworm), and L

In [28]:
from pprint import pprint

In [29]:
pprint(str(response))

('Arthropods are the largest phylum of Animalia which includes insects. They '
 'have an organ-system level of organisation, are bilaterally symmetrical, '
 'triploblastic, segmented and coelomate animals. Their body is covered by a '
 'chitinous exoskeleton and consists of head, thorax and abdomen. They have '
 'jointed appendages, respiratory organs, and sensory organs like antennae, '
 'eyes, and balancing organs. Excretion takes place through malpighian tubules '
 'and they are mostly dioecious. Fertilisation is usually internal and they '
 'are mostly oviparous. Examples of economically important arthropods include '
 'Apis (Honey bee), Bombyx (Silkworm), and Laccifer (Lac insect).')
